In [5]:
include("Create_Sample.jl")
include("MIP_SPLIT.jl")
include("Split.jl")
include("GA.jl")
include("Initial.jl")
include("Mutation.jl")
include("Crossover.jl")
include("Neighborhood.jl")
include("Neighborhood_intra.jl")
include("costs.jl")

Calculate_new_cost_3_permute (generic function with 1 method)

In [6]:
instances = [:eil51, :berlin52, :eil76, :rat99]
LKH3 = [[222.7, 159.6, 124.0, 112.1],[4110.2, 3184.2, 2440.9, 2440.9], [280.9, 196.7, 143.4, 128.2],[690.8, 523.3, 467.0,442.5]]
Ms = [2,3,5,7]
max_infs = [1,2,3]
# best_v = Float64[]
# Avg_v = Float64[]
# worst_v = Float64[]

for max_inf in max_infs
    count = 0
    best_ = 0.0
    Avg_ = 0.0
    worst_ = 0.0
    time_ = 0.0
    println("maximum infeasible: ", max_inf)
    for (i,instance) in enumerate(instances)
        for (j,K) in enumerate(Ms)
            count += 1
            T = Read_TSPLIB_instance(instance, 1)
            n = size(T)[1]-2
            demands = ones(Int, n)
            W = 150
            h = 0.3
            num_nei = 2
            popsize = (20,50)
            k_tournament = 3
            num_iter = 2000
            Mutation_Chance = 0.0
            num_runs = 10
            targetFeasible = 0.2
            avg = 0.0
            best = Inf
            worst = 0.0
            t1 = time() 
            P = Chromosome[]
            for i=1:num_runs
                P = Perform_Genetic_Algorithm(T, demands,K, W, h, popsize, k_tournament, num_iter, Mutation_Chance, num_nei, targetFeasible, max_inf);
                solution = best_objective(P)
                avg += solution
                if solution < best
                    best = solution
                end
                if solution > worst
                    worst = solution
                end
            end
            t2 = time()
            println("Results for ", instance, " ,m=", K)
            println("Best: ", round(best, digits = 1), "  Average: ", round(avg/num_runs, digits = 1), 
                "  Worst: ", round(worst, digits = 1), " , run time= ", round((t2-t1)/num_runs, digits=0))
            best_ += 100*(best-LKH3[i][j])/LKH3[i][j]
            worst_ += 100*(worst-LKH3[i][j])/LKH3[i][j]
            Avg_ += 100*(avg/num_runs-LKH3[i][j])/LKH3[i][j]
            time_ += (t2-t1)/num_runs
        end
    end
    #     push!(best_v, best_/count)
    #     push!(worst_v, worst_/count)
    #     push!(Avg_v, Avg_/count)
    println("Overall Results:  Best: ", round(best_/count, digits=1), "  Average: ", round(Avg_/count, digits=1), "  Worst: ", round(worst_/count, digits=1), "   run time: ", round(time_/count, digits=1))
end
    

maximum infeasible: 1
Results for eil51 ,m=2
Best: 225.2  Average: 230.9  Worst: 239.0 , run time= 1.0
Results for eil51 ,m=3
Best: 162.0  Average: 169.7  Worst: 174.2 , run time= 2.0
Results for eil51 ,m=5
Best: 122.7  Average: 125.1  Worst: 127.5 , run time= 3.0
Results for eil51 ,m=7
Best: 112.1  Average: 112.1  Worst: 112.5 , run time= 1.0
Results for berlin52 ,m=2
Best: 4206.7  Average: 4242.5  Worst: 4270.7 , run time= 1.0
Results for berlin52 ,m=3
Best: 3189.6  Average: 3196.4  Worst: 3212.4 , run time= 2.0
Results for berlin52 ,m=5
Best: 2440.9  Average: 2466.7  Worst: 2485.6 , run time= 2.0
Results for berlin52 ,m=7
Best: 2440.9  Average: 2440.9  Worst: 2440.9 , run time= 1.0
Results for eil76 ,m=2
Best: 290.8  Average: 294.9  Worst: 300.8 , run time= 2.0
Results for eil76 ,m=3
Best: 201.3  Average: 212.2  Worst: 221.0 , run time= 5.0
Results for eil76 ,m=5
Best: 152.7  Average: 156.5  Worst: 161.4 , run time= 8.0
Results for eil76 ,m=7
Best: 132.1  Average: 135.6  Worst: 140.

In [119]:
for tour in P[1].tours
    t1 = copy(tour.Sequence);
    pushfirst!(t1, 0)
    push!(t1, n+1)

    z1 = 0.0
    for i=1:length(t1)-1
        z1 += T[t1[i]+1, t1[i+1]+1]
    end

    println(z1, " ,  ", tour.cost)
end

LoadError: BoundsError: attempt to access 0-element Vector{Chromosome} at index [1]

In [83]:
function Calculate_new_cost_3_permute1(tour::Vector{Int}, cost::Float64, S1::Vector{Int}, S2::Vector{Int}
        ,k1::Int, T::Matrix{Float64}, n_nodes::Int)
    if S1 == S2 
        return cost
    end
    nt = length(tour)
    cost = cost - T[S1[1]+1, S1[2]+1] - T[S1[2]+1, S1[3]+1] + T[S2[1]+1, S2[2]+1] + T[S2[2]+1, S2[3]+1]
    
    if k1 == 1
        cost = cost - T[1, S1[1]+1] + T[1, S2[1]+1] - T[S1[3]+1, tour[4]+1] + T[S2[3]+1, tour[4]+1] 
    elseif k1 == nt-2
        cost = cost - T[S1[3]+1, n_nodes+2] + T[S2[3]+1, n_nodes+2] - T[tour[nt-3]+1, S1[1]+1] + T[tour[nt-3]+1, S2[1]+1]
    else
        cost = cost - T[tour[k1-1]+1, S1[1]+1] + T[tour[k1-1]+1, S2[1]+1] - T[S1[3]+1, tour[k1+3]+1] + T[S2[3]+1, tour[k1+3]+1] 
    end
    return cost
end



function Calculate_new_cost_3_permute2(tour::Vector{Int}, cost::Float64, S1::Vector{Int}, S2::Vector{Int}
        ,k1::Int, T::Matrix{Float64}, n_nodes::Int)

    nt = length(tour)
    
    t1 = copy(tour)
    t1[k1:k1+2] = S2
    pushfirst!(t1, 0)
    push!(t1, n_nodes+1)
    z1 = 0.0
    for i=1:length(t1)-1
        z1 += T[t1[i]+1, t1[i+1]+1]
    end
    
    return z1
end

Calculate_new_cost_3_permute2 (generic function with 1 method)

In [38]:
instance = :rat99
T = Read_TSPLIB_instance(instance, 1)
n_nodes = size(T)[1]-2
tsp_tour = find_tsp_tour1(T)
demands = ones(Int, n_nodes)
W = 150
K = 2
mu = 10
P, _ = Generate_initial_population(T, demands, K, W, mu, tsp_tour);

In [39]:
P[1].tours

2-element Vector{Tour}:
 Tour([9, 10, 11, 12, 21, 20, 19, 18, 27, 36  …  82, 90, 91, 92, 93, 94, 95, 96, 97, 98], 823.6584356071808)
 Tour([89, 88, 78, 87, 86, 85, 84, 83, 75, 76  …  17, 8, 7, 6, 5, 4, 13, 3, 2, 1], 806.3918274353607)

In [74]:
tour1 = copy(P[1].tours[1].Sequence)
tour2 = copy(P[1].tours[1].Sequence)
cost1 = P[1].tours[1].cost
cost2 = P[1].tours[1].cost



823.6584356071808

In [85]:
include("Neighborhood.jl")

tt1 = 0.0
tt2 = 0.0
t1 = time()
for i=1:10000000
    k1 = rand(1:length(tour1)-2)
    S1 = copy(tour1[k1:k1+2])
    S2 = shuffle(S1)

    new_cost1 = Calculate_new_cost_3_permute1(tour1, cost1, S1, S2, k1, T, n_nodes)
#     z1 = Calculate_new_cost_3_permute2(tour1, cost1, S1, S2, k1, T, n_nodes)

#     if round(new_cost1, digits = 6) != round(z1, digits = 6)
#         println(k1)
#         println(round(new_cost1, digits = 6), "   ", round(z1, digits = 6))
#     end
end

println("time for 1:", time()- t1)

t2 = time()
for i=1:10000000
    k1 = rand(1:length(tour1)-2)
    S1 = copy(tour1[k1:k1+2])
    S2 = shuffle(S1)

    z1 = Calculate_new_cost_3_permute2(tour1, cost1, S1, S2, k1, T, n_nodes)

end

println("time for 2:", time()- t2)


time for 1:4.017000198364258
time for 2:9.663999795913696


In [42]:
length(tour1)

49

In [2]:
function find_neighbors(a::Int, i::Int, m::Int)
    
    neighbors = Int[]
    for j=max(1, i-m):i-1
        push!(neighbors, j)
    end
    for j=i+1:min(a, i+m)
        push!(neighbors, j)
    end
    return neighbors
end

find_neighbors (generic function with 1 method)

In [5]:
find_neighbors(20, 4, 50)

19-element Vector{Int64}:
  1
  2
  3
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17
 18
 19
 20

In [68]:
a == b

true